In [4]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge
import Outils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    df = gp.read_postgis("select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom from public.traf2015_bdt17_ed15_l t left join public.traf2015_bdt17_ed15_l_vertices_pgr v1 on t.\"source\"=v1.id left join public.traf2015_bdt17_ed15_l_vertices_pgr v2  on t.target=v2.id", c.connexionPsy)
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df.loc[:,'nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

In [77]:
#affectation des lignes aux rd points
df=at.identifier_rd_pt(df)

loongueur l_dans_p : 4074 
loongueur l_dans_p_int : 2 
loongueur l_dans_p_final : 4071


In [525]:
df_ligne=df.loc['TRONROUT0000000032845684']
df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne['target']) | (df['target'] == df_ligne['target']))]
liste_ligne_touchees=df_touches_source.index.tolist()

In [536]:
(df_touches_source['nature']=='Rd_pt').all()

True

In [114]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000032862365',df,[]))

In [115]:
toto.append('TRONROUT0000000032862365')

In [116]:
toto

['TRONROUT0000000032862368',
 'TRONROUT0000000032864987',
 'TRONROUT0000000032866357',
 'TRONROUT0000000032867486',
 'TRONROUT0000000032867484',
 'TRONROUT0000000032867501',
 'TRONROUT0000000032869886',
 'TRONROUT0000000032869892',
 'TRONROUT0000000032872322',
 'TRONROUT0000000032872324',
 'TRONROUT0000000032872329',
 'TRONROUT0000000032876521',
 'TRONROUT0000000032880628',
 'TRONROUT0000000032880629',
 'TRONROUT0000000032882244',
 'TRONROUT0000000032882262',
 'TRONROUT0000000032885211',
 'TRONROUT0000000032885215',
 'TRONROUT0000000032885213',
 'TRONROUT0000000032887582',
 'TRONROUT0000000032887587',
 'TRONROUT0000000032887590',
 'TRONROUT0000000032887598',
 'TRONROUT0000000032889546',
 'TRONROUT0000000032893105',
 'TRONROUT0000000032893109',
 'TRONROUT0000000032895058',
 'TRONROUT0000000032862365']

In [117]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


'TRONROUT0000000032876496'

In [ ]:
for i in toto :
    tata=at.affecter_troncon_ligne(i,df,df2_chaussees)
    print(len(tata))

In [ ]:
df2_chaussees

In [6]:
dico=at.affecter_troncon(df,df2_chaussees)
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

0eme occurence : TRONROUT0000000084053027 à 14:55:16 nb ligne traite : 0, nb ligne differente=0


C:\Users\Martin\git\otv\otv\Transfert_Donnees\agrege_troncon.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


indexError ligne  TRONROUT0000000032954220
indexError ligne  TRONROUT0000000033016282
indexError ligne  TRONROUT0000000032859646
indexError ligne  TRONROUT0000000032836415


POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [87]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [88]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')
gdf_lignes.to_file(r'D:\temp\otv\17\2_chaussees\lignes_a_gouper.shp')

In [89]:
#union des geometries
gdf_lignes2=linemerge(gdf_lignes.unary_union)

In [ ]:
gdf_lignes2.wkt

In [90]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds
point=gdf_lignes2.interpolate(0.5, normalized=True)

In [91]:
point.wkt

'POINT (385035.1182861876 6561685.156661048)'

In [92]:
#recherche des lignes proches du centroid
lignes_possibles=df.cx[xmin:xmax,ymin:ymax]
lignes_possibles.to_file(r'D:\temp\otv\17\2_chaussees\lignes_possibles.shp')

In [93]:
lignes_possibles

,geom,id,nature,numero,importance,cl_admin,gestion,mise_serv,fictif,franchisst,...,codevoie_d,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt,codevoie_rdpt,assigne_rdpt
TRONROUT0000000032869330,(LINESTRING (385085.3999999997 6561605.4999999...,40964,Bretelle,D137C0,3,Départementale,17,NR,Non,NC,...,NR,3,POINT(385085.4 6561605.49999999),3,POINT(385210.2 6561785.99999999),NaN,NaN,NaN,NaN,None
TRONROUT0000000129676311,(LINESTRING (385040.3999999997 6561674.3999999...,214445,Quasi-autoroute,D137,1,Départementale,17,NR,Non,NC,...,NR,2,POINT(385040.4 6561674.39999999),3,POINT(385109.2 6561527.59999999),NaN,NaN,NaN,NaN,None
TRONROUT0000000129676313,(LINESTRING (385048.2999999996 6561682.3999999...,214447,Quasi-autoroute,D137,1,Départementale,17,NR,Non,NC,...,NR,2,POINT(385048.3 6561682.39999999),3,POINT(384951.3 6561896.59999999),NaN,NaN,NaN,NaN,None
TRONROUT0000000032869425,(LINESTRING (384973.7999999997 6561648.0999999...,41050,Bretelle,D137C0,3,Départementale,17,NR,Non,NC,...,NR,3,POINT(384973.8 6561648.09999999),3,POINT(385109.2 6561527.59999999),NaN,NaN,NaN,NaN,None
TRONROUT0000000032869424,(LINESTRING (384962.7999999996 6561843.4999999...,41049,Bretelle,D137C0,3,Départementale,17,NR,Non,NC,...,NR,3,POINT(384962.8 6561843.49999999),3,POINT(384973.8 6561648.09999999),NaN,NaN,NaN,NaN,None
TRONROUT0000000129676312,(LINESTRING (385085.3999999997 6561605.4999999...,214446,Quasi-autoroute,D137,1,Départementale,17,NR,Non,NC,...,NR,3,POINT(385085.4 6561605.49999999),2,POINT(385048.3 6561682.39999999),NaN,NaN,NaN,NaN,None
TRONROUT0000000129676310,(LINESTRING (385040.3999999997 6561674.3999999...,214444,Quasi-autoroute,D137,1,Départementale,17,NR,Non,NC,...,NR,2,POINT(385040.4 6561674.39999999),3,POINT(384962.8 6561843.49999999),NaN,NaN,NaN,NaN,None


In [94]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [95]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
#garder uniquement la valeur la plus proche du centroid
ligne_proche=ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index.tolist()[0]
ligne_proche

'TRONROUT0000000129676313'

POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [15]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14] # car un cercle à un rappor de ce type entre 12 et 13
dico_rd_pt=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point=gp.GeoDataFrame(dico_rd_pt, geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'facteur','geometry']
#on créer aussi la meme donnees avec un buffer interiuer, pour ne garder que les lignes dans le buffer exteriuer et hors buffer interieur (cas de rond point enjmabeant uine 2*2 et prenant la 2*2 voie qui est dans le polygone
liste_rd_points_int=[t.buffer(-0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt_int=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point_int=gp.GeoDataFrame(dico_rd_pt_int, geometry=liste_rd_points_int)
gdf_rd_point_int.crs={'init':'epsg:2154'}
gdf_rd_point_int.columns=['id_rdpt', 'facteur','geometry']

In [63]:
#jointure spataile pour une gdf avec uniquement les lignes des rd_points avec le numéro
l_dans_p=gp.sjoin(df,gdf_rd_point,op='within') 

In [64]:
#suprimer les doublons
l_dans_p_ss_doublon=l_dans_p.reset_index().rename(columns={'index':'id_ign'}).drop_duplicates('id_ign').set_index('id_ign')

In [66]:
l_dans_p_int=gp.sjoin(df,gdf_rd_point_int,op='within')

In [67]:
l_dans_p_final=l_dans_p_ss_doublon.loc[~l_dans_p_ss_doublon.index.isin(l_dans_p_int.index.tolist())]

In [68]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df,l_dans_p_final.drop('index_right', axis=1), how='inner',op='intersects')

In [69]:
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p_final.index.tolist())][['id_rdpt','numero_left','codevoie_d_left']]

In [70]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x))),
    l_intersct_rdpt.groupby('id_rdpt')['codevoie_d_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt','codevoie_rdpt']

In [ ]:
l_dans_p_final

In [297]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df.drop('id_rdpt',axis=1),l_dans_p_final.loc[:,'id_rdpt']],axis=1, sort=False)

In [298]:
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)

In [299]:
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)

In [320]:
def assigne_rdpt(id_rdpt,numero, liste_num, codevoie, liste_code) : 
    if id_rdpt >=0 :
        if (numero in liste_num) or (codevoie in liste_code) :
            return True
        else : 
            return False

In [321]:
gdf_avec_rd_pt['assigne_rdpt']=gdf_avec_rd_pt.apply(lambda x : assigne_rdpt(x['id_rdpt'],x['numero'], x['nom_rte_rdpt_y'], x['codevoie_d'],x['codevoie_rdpt']),axis=1)

In [ ]:
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==l_dans_p['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

In [78]:
#pour export et verif uniquement

#gdf_l_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test2.shp')
#gdf_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test1.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
df.to_file(r'D:\temp\otv\17\test_rd_point\test7.shp')
#df.to_file(r'D:\temp\otv\17\test_rd_point\test5.shp')

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [91]:
','.join(set(['a','a','b','c']))

'b,c,a'

In [228]:
id_ign_ligne='TRONROUT0000000342862858'
df_ligne = df.loc[id_ign_ligne]
ligne_traite_troncon=[]
ligne_traite_troncon.append(id_ign_ligne)
liste_ligne_suivantes=[] 
liste_ligne_touchees=[]

In [284]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() :
    df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne[value[0]]) | (df['target'] == df_ligne[value[0]]))]
    df_rd_pt=df.loc[df['id_rdpt']==df_touches_source.iloc[0]['id_rdpt']]
    l_df_rd_pt=df.loc[df.index.isin(gp.sjoin(df,df_rd_pt, how='inner',op='intersects').index.unique().tolist())]
    l_df_rd_pt2=l_df_rd_pt.loc[~l_df_rd_pt.index.isin(df_rd_pt.index.tolist())].index.unique().tolist()
    l_df_rd_pt3=df.loc[df.index.isin(l_df_rd_pt2),'codevoie_d'].unique().tolist()
    print(l_df_rd_pt3)
    #liste_ligne_touchees+=df_touches_source.index.tolist()

['173006945', '173003140', '173006035']


RTreeError: Error in "Index_CreateWithStream": IllegalArgumentException: RTree::BulkLoader::bulkLoadUsingSTR: Empty data stream given.

In [233]:
df_touches_source['id_rdpt'].values.tolist()

[nan]

In [231]:
liste_ligne_touchees

['TRONROUT0000000032845699',
 'TRONROUT0000000032845656',
 'TRONROUT0000000032845688']

In [440]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() : 
    print(key, value)

nb_intrsct_src ['source', 'src_geom']
nb_intrsct_tgt ['target', 'tgt_geom']


In [478]:
for key, value in [['a','b','c'],['d','e','f']].items() :
    print(key, value)

AttributeError: 'list' object has no attribute 'items'

In [568]:
for a in []:
    print(a)